In [1]:
import cmath
import numpy as np
import math as m
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [2]:
# === Constantes physiques ===
k = 2 * m.pi * 40000 / 343
# R_p = 0.0005 ##rayon particule finale
R_p = 0.0015 # Rayon des objets actuellement utilisés
R_t=0.005 ##rayon transducteur
c_p = 2400
rho_p = 28.59
c_0 = 343
rho_0 = 1.204
eps = 1e-4

# === Grille de transducteurs ===
a = 0.01
n = 8
l = np.linspace(-(n // 2 - 0.5) * a, (n // 2 - 0.5) * a, n)
X_j, Y_j = np.meshgrid(l, l)
X_j = X_j.flatten()
Y_j = Y_j.flatten()
Z_j = np.zeros_like(X_j)

# === Fonctions utiles ===

def K1(R, c_p, rho_p):
    V = 4 * m.pi / 3 * R**3
    a0 = 1 / (c_0**2 * rho_0)
    ap = 1 / (c_p**2 * rho_p)
    return V / 4 * (a0 - ap)

def K2(R, omega, rho_p):
    V = 4 * m.pi / 3 * R**3
    return 3 * V / 4 * ((rho_0 - rho_p) / (omega**2 * rho_0 * (rho_0 + 2 * rho_p)))


def D_F_vectorized(xj, yj, zj, x, y, z):
    theta = np.arctan(np.sqrt((x - xj)**2 + (y - yj)**2) / np.where(np.abs(z - zj) < 1e-12, 1e-12, np.abs(z - zj)))
    df = np.sin(k * R_p * np.sin(theta)) / np.where(np.sin(k * R_p * np.sin(theta)) == 0, 1e-12, k * R_p * np.sin(theta))
    df = np.where(np.isnan(df), 1, df)
    return df

# === Pré-calculs (ne dépendent pas des phases) ===
DJ={}
DF={}
def precompute_field_params(x, y, z):
    # Plaque du bas
    if (x,y,z) in DJ:
        return (DJ[(x,y,z)],DF[(x,y,z)])

    else :
      dx = X_j - x
      dy = Y_j - y
      dz = Z_j - z
      dj = np.sqrt(dx**2 + dy**2 + dz**2)
      df = D_F_vectorized(X_j, Y_j, Z_j, x, y, z)
      DJ[(x,y,z)]=dj
      DF[(x,y,z)]=df

      return (dj, df)

# === Calcul du champ total ===

def p_tot(x, y, z, phi_vals, dj, df):
    exp_term = np.exp(1j * (k * dj + phi_vals))
    return np.sum(df / dj * exp_term)

# === Gradient numérique du champ ===

def grad_p(x, y, z, phi_vals, dj0, df0):
    def shifted_p(dx, dy, dz):
        dj, df = precompute_field_params(x + dx, y + dy, z + dz)
        return p_tot(x + dx, y + dy, z + dz, phi_vals, dj, df)

    dp_dx = (shifted_p(eps, 0, 0) - shifted_p(0, 0, 0)) / eps
    dp_dy = (shifted_p(0, eps, 0) - shifted_p(0, 0, 0)) / eps
    dp_dz = (shifted_p(0, 0, eps) - shifted_p(0, 0, 0)) / eps
    return np.array([dp_dx, dp_dy, dp_dz])

# === Potentiel de Gorkov ===

def gorkov_potential(x, y, z, phi_vals, dj, df):
    p_complex = p_tot(x, y, z, phi_vals, dj, df)
    gradp = grad_p(x, y, z, phi_vals, dj, df)

    omega = 2 * np.pi * 40000
    k1 = K1(R_p, c_p, rho_p)
    k2 = K2(R_p, omega, rho_p)

    abs_p2 = abs(p_complex)**2
    abs_gradp2 = np.sum(np.abs(gradp)**2)
    return k1 * abs_p2 - k2 * abs_gradp2

# === Laplacien du potentiel ===

def lap_U(x, y, z, phi_vals, dj0, df0):
    def U_shift(dx, dy, dz):
        dj, df = precompute_field_params(x + dx, y + dy, z + dz)
        return gorkov_potential(x + dx, y + dy, z + dz, phi_vals, dj, df)

    dU_dxx = (U_shift(eps, 0, 0) - 2*U_shift(0, 0, 0) + U_shift(-eps, 0, 0)) / eps**2
    dU_dyy = (U_shift(0, eps, 0) - 2*U_shift(0, 0, 0) + U_shift(0, -eps, 0)) / eps**2
    dU_dzz = (U_shift(0, 0, eps) - 2*U_shift(0, 0, 0) + U_shift(0, 0, -eps)) / eps**2
    return dU_dxx + dU_dyy + dU_dzz

# === Fonction objectif ===

def F_opt(xf, yf, zf, phi_vals, dj, df):
    ptot = p_tot(xf, yf, zf, phi_vals, dj, df)
    lapU = lap_U(xf, yf, zf, phi_vals, dj, df)
    return abs(ptot) - lapU



In [3]:
# === Optimiseur ===

def optimizer2(xf, yf, zf):
    dj, df = precompute_field_params(xf, yf, zf)
    def objective(phi_vals):
        phi_mod = np.mod(phi_vals, 2*np.pi)
        return F_opt(xf, yf, zf, phi_mod, dj, df)

    phi0 = np.zeros(n**2)
    result = minimize(objective, phi0, method='L-BFGS-B')
    phases_mod = np.mod(result.x, 2*np.pi)
    print("Phases optimales trouvées (mod 2π) :")
    print(", ".join(f"{a:.4f}" for a in phases_mod))
    return phases_mod

# === Point de focalisation ===
X_foc, Y_foc, Z_foc = 0, 0, 0.03

# === Lancement ===
optimizer2(X_foc, Y_foc, Z_foc)

Phases optimales trouvées (mod 2π) :
0.1606, 0.0266, 0.0372, 5.9550, 5.9550, 0.0372, 0.0266, 0.1606, 0.0266, 5.9550, 0.3303, 6.1748, 6.1748, 0.3303, 5.9550, 0.0266, 0.0372, 0.3303, 5.9318, 0.3740, 0.3740, 5.9318, 0.3303, 0.0372, 5.9550, 6.1748, 0.3740, 6.1393, 6.1393, 0.3740, 6.1748, 5.9550, 5.9550, 6.1748, 0.3740, 6.1393, 6.1393, 0.3740, 6.1748, 5.9550, 0.0372, 0.3303, 5.9318, 0.3740, 0.3740, 5.9318, 0.3303, 0.0372, 0.0266, 5.9550, 0.3303, 6.1748, 6.1748, 0.3303, 5.9550, 0.0266, 0.1606, 0.0266, 0.0372, 5.9550, 5.9550, 0.0372, 0.0266, 0.1606


array([0.16057536, 0.02658712, 0.03717466, 5.95503812, 5.95503812,
       0.03717475, 0.02658713, 0.1605754 , 0.02658713, 5.95503812,
       0.33026945, 6.17476676, 6.17476671, 0.33026945, 5.95503813,
       0.02658713, 0.03717471, 0.33026945, 5.93175305, 0.37400245,
       0.37400245, 5.93175306, 0.33026945, 0.0371748 , 5.95503812,
       6.17476666, 0.37400246, 6.13925298, 6.13925304, 0.3740024 ,
       6.17476677, 5.95503813, 5.95503813, 6.17476671, 0.37400245,
       6.13925304, 6.13925304, 0.3740024 , 6.17476676, 5.95503814,
       0.03717476, 0.33026941, 5.93175306, 0.3740024 , 0.3740024 ,
       5.93175303, 0.33026944, 0.03717481, 0.02658708, 5.95503813,
       0.33026945, 6.17476676, 6.17476676, 0.33026945, 5.95503812,
       0.02658705, 0.1605754 , 0.02658708, 0.03717476, 5.95503814,
       5.95503813, 0.0371748 , 0.02658704, 0.16057544])